# Balance Sheet Display Demo

This notebook demonstrates the balance sheet visualization capabilities in bilancio, including the new deliverable valuation feature.

In [52]:
# Import required modules
from decimal import Decimal
from bilancio.engines.system import System
from bilancio.domain.agents.bank import Bank
from bilancio.domain.agents.central_bank import CentralBank
from bilancio.domain.agents.household import Household
from bilancio.domain.agents.treasury import Treasury
from bilancio.analysis.balances import agent_balance, system_trial_balance
from bilancio.analysis.visualization import (
    display_agent_balance_table,
    display_agent_balance_from_balance,
    display_multiple_agent_balances
)
from bilancio.ops.banking import deposit_cash, withdraw_cash

## 1. Setup System and Agents

In [53]:
# Initialize the system
system = System()

# Create central bank
central_bank = CentralBank(id="cb", name="Federal Reserve", kind="central_bank")
system.bootstrap_cb(central_bank)

# Create other agents
bank = Bank(id="bank1", name="Community Bank", kind="bank")
hh1 = Household(id="hh1", name="The Smiths", kind="household")
hh2 = Household(id="hh2", name="The Johnsons", kind="household")
treasury = Treasury(id="treasury", name="US Treasury", kind="treasury")

# Add agents
system.add_agents([bank, hh1, hh2, treasury])

print(f"System created with {len(system.state.agents)} agents")

System created with 5 agents


## 2. Create Financial Instruments

In [54]:
# Central bank provides reserves to bank
system.mint_reserves(to_bank_id="bank1", amount=10000)

# Issue cash to households
system.mint_cash(to_agent_id="hh1", amount=1000)
system.mint_cash(to_agent_id="hh2", amount=1500)

# Bank converts some reserves to cash
system.convert_reserves_to_cash(bank_id="bank1", amount=5000)

# Households deposit cash at bank
deposit_cash(system, customer_id="hh1", bank_id="bank1", amount=800)
deposit_cash(system, customer_id="hh2", bank_id="bank1", amount=1200)

print("Financial instruments created")

Financial instruments created


## 3. Create Deliverables with Required Monetary Values

All deliverables now REQUIRE a unit_price parameter. This ensures balance sheets always show proper monetary values, solving the issue of mixing monetary values with physical quantities.

In [ ]:
# All deliverables now REQUIRE a monetary value (no more unvalued items)
# Create a valued deliverable (lawn service worth $50)
system.create_deliverable(
    issuer_id="hh2",
    holder_id="hh1",
    sku="LAWN_SERVICE",
    quantity=1,
    unit_price=Decimal("50.00")  # $50 per lawn service - REQUIRED
)

# Create a zero-priced deliverable (free cookies)
system.create_deliverable(
    issuer_id="hh1",
    holder_id="hh2",
    sku="COOKIES",
    quantity=12,
    unit_price=Decimal("0")  # Free items must explicitly have price=0
)

# Create another valued deliverable (groceries)
system.create_deliverable(
    issuer_id="treasury",
    holder_id="hh1",
    sku="GROCERIES",
    quantity=5,
    unit_price=Decimal("10.00")  # $10 per unit
)

print("Deliverables created with required monetary values")

## 4. Display Single Agent Balance Sheet

In [56]:
# Display bank's balance sheet
print("Community Bank Balance Sheet:\n")
display_agent_balance_table(system, "bank1")

Community Bank Balance Sheet:



                                      Community Bank (bank)                                      
╭────────────────────────────────┬──────────────┬────────────────────────────────┬──────────────╮
│ ASSETS                         │       Amount │ LIABILITIES                    │       Amount │
├────────────────────────────────┼──────────────┼────────────────────────────────┼──────────────┤
│ cash                           │        7,000 │ bank_deposit                   │        2,000 │
│ reserve_deposit                │        5,000 │                                │              │
│                                │              │                                │              │
├────────────────────────────────┼──────────────┼────────────────────────────────┼──────────────┤
│ TOTAL FINANCIAL                │       12,000 │ TOTAL FINANCIAL                │        2,000 │
│                                │              │                                │              │
├────────────────────────────────┼──────────────┼────────────────────────────────┼──────────────┤
│                                │              │ NET FINANCIAL                  │      +10,000 │
╰────────────────────────────────┴──────────────┴────────────────────────────────┴──────────────╯

## 5. Examine Deliverable Valuation Details

In [57]:
# Get household 1 balance to see valued deliverables
hh1_balance = agent_balance(system, "hh1")

print("The Smiths' Balance Analysis:")
print(f"\nFinancial Assets: ${hh1_balance.total_financial_assets:,}")
print(f"Non-Financial Asset Value: ${hh1_balance.total_nonfinancial_value or 0}")
print(f"Total Assets: ${hh1_balance.total_financial_assets + (hh1_balance.total_nonfinancial_value or 0)}")

print("\nNon-Financial Assets Detail:")
for sku, details in hh1_balance.nonfinancial_assets_by_kind.items():
    print(f"  {sku}:")
    print(f"    Quantity: {details['quantity']} units")
    if details['value'] is not None:
        print(f"    Value: ${details['value']} (included in totals)")
    else:
        print(f"    Value: Not set (excluded from totals)")

The Smiths' Balance Analysis:

Financial Assets: $1,000
Non-Financial Asset Value: $100.00
Total Assets: $1100.00

Non-Financial Assets Detail:
  LAWN_SERVICE:
    Quantity: 1 units
    Value: $50.00 (included in totals)
  GROCERIES:
    Quantity: 5 units
    Value: $50.00 (included in totals)


## 6. Compare Household Balance Sheets

Notice how deliverables now show both quantity (in brackets) and monetary value properly.

In [ ]:
# Compare both households
print("Household Balance Sheets Comparison:\n")
display_multiple_agent_balances(system, ["hh1", "hh2"], format='simple')

print("\nKey Points:")
print("- All deliverables show monetary amounts (required unit_price)")
print("- Free items explicitly show $0 value")
print("- Quantities shown in brackets for context")
print("- Balance sheets always show proper monetary totals!")

## 7. Update Deliverable Pricing

In [59]:
# Find a deliverable to update (cookies)
cookie_id = None
for contract_id, contract in system.state.contracts.items():
    if getattr(contract, 'sku', None) == 'COOKIES' and contract.asset_holder_id == 'hh2':
        cookie_id = contract_id
        break

if cookie_id:
    print("Before pricing cookies:")
    hh2_balance = agent_balance(system, "hh2")
    print(f"Total non-financial value: ${hh2_balance.total_nonfinancial_value or 0}")
    
    # Now price the cookies
    system.update_deliverable_price(cookie_id, Decimal("2.50"))  # $2.50 per cookie
    
    print("\nAfter pricing cookies at $2.50 each:")
    hh2_balance = agent_balance(system, "hh2")
    print(f"Total non-financial value: ${hh2_balance.total_nonfinancial_value or 0}")
    print(f"Cookie value: 12 × $2.50 = $30")

Before pricing cookies:
Total non-financial value: $0

After pricing cookies at $2.50 each:
Total non-financial value: $30.00
Cookie value: 12 × $2.50 = $30


## 8. System-Wide Trial Balance

In [60]:
# Get system trial balance
trial_balance = system_trial_balance(system)

print("System-Wide Trial Balance:")
print("="*50)

print("\nFinancial Instruments:")
print(f"  Total Financial Assets: ${trial_balance.total_financial_assets:,}")
print(f"  Total Financial Liabilities: ${trial_balance.total_financial_liabilities:,}")

print("\nNon-Financial Assets:")
print(f"  Total Valued Deliverables: ${trial_balance.total_nonfinancial_value or 0}")

print("\nDeliverable Details:")
for sku, details in trial_balance.nonfinancial_assets_by_kind.items():
    value_str = f"${details['value']}" if details['value'] is not None else "unvalued"
    print(f"  {sku}: {details['quantity']} units ({value_str})")

# Verify double-entry bookkeeping
if trial_balance.total_financial_assets == trial_balance.total_financial_liabilities:
    print("\n✅ Financial system is in balance")
else:
    print("\n❌ Financial system is NOT in balance!")

System-Wide Trial Balance:

Financial Instruments:
  Total Financial Assets: $14,500
  Total Financial Liabilities: $14,500

Non-Financial Assets:
  Total Valued Deliverables: $130.00

Deliverable Details:
  LAWN_SERVICE: 1 units ($50.00)
  COOKIES: 12 units ($30.00)
  GROCERIES: 5 units ($50.00)

✅ Financial system is in balance


## 9. Display All Agent Balance Sheets

In [61]:
# Display all agents
print("Complete System Overview:\n")
all_agent_ids = list(system.state.agents.keys())
display_multiple_agent_balances(system, all_agent_ids)

Complete System Overview:



       Federal Reserve                         Community Bank                           The Smiths                 
        (central_bank)                             (bank)                              (household)                 
╭────────────────┬───────────╮         ╭────────────────┬───────────╮         ╭────────────────┬───────────╮       
│ Item           │    Amount │         │ Item           │    Amount │         │ Item           │    Amount │       
├────────────────┼───────────┤         ├────────────────┼───────────┤         ├────────────────┼───────────┤       
│ ASSETS         │           │         │ ASSETS         │           │         │ ASSETS         │           │       
│                │           │         │ cash           │     7,000 │         │ bank_deposit   │       800 │       
├────────────────┼───────────┤         │ reserve_depos… │     5,000 │         │ cash           │       200 │       
│ LIABILITIES    │           │         │                │           │         │ deliverable... │        50 │       
│ cash           │     7,500 │         ├────────────────┼───────────┤         │ deliverable... │        50 │       
│ reserve_depos… │     5,000 │         │ LIABILITIES    │           │         │                │           │       
│                │           │         │ bank_deposit   │     2,000 │         ├────────────────┼───────────┤       
├────────────────┼───────────┤         │                │           │         │ LIABILITIES    │           │       
│ Total          │         0 │         ├────────────────┼───────────┤         │ deliverable    │        12 │       
│ Financial      │           │         │ Total          │    12,000 │         │                │           │       
│ Total Liab.    │    12,500 │         │ Financial      │           │         ├────────────────┼───────────┤       
│ Net Financial  │   -12,500 │         │ Total Liab.    │     2,000 │         │ Total          │     1,000 │       
╰────────────────┴───────────╯         │ Net Financial  │   +10,000 │         │ Financial      │           │       
                                       ╰────────────────┴───────────╯         │ Total Valued   │       100 │       
                                                                              │ Total Assets   │     1,100 │       
                                                                              │ Total Liab.    │         0 │       
                                                                              │ Net Financial  │    +1,000 │       
                                                                              ╰────────────────┴───────────╯       
         The Johnsons                           US Treasury                                                        
         (household)                             (treasury)                                                        
╭────────────────┬───────────╮         ╭────────────────┬───────────╮                                              
│ Item           │    Amount │         │ Item           │    Amount │                                              
├────────────────┼───────────┤         ├────────────────┼───────────┤                                              
│ ASSETS         │           │         │ ASSETS         │           │                                              
│ bank_deposit   │     1,200 │         │                │           │                                              
│ cash           │       300 │         ├────────────────┼───────────┤                                              
│ deliverable... │        30 │         │ LIABILITIES    │           │                                              
│                │           │         │ deliverable    │         5 │                                              
├────────────────┼───────────┤         │                │           │                                              
│ LIABILITIES    │           │         ├────────────────

## Summary

### Key Features Demonstrated:

1. **Required Deliverable Valuation** - All deliverables MUST have a unit_price (use 0 for free items)
2. **Proper Monetary Accounting** - Balance sheets always show monetary values, never raw quantities
3. **Clear Quantity Display** - Physical quantities shown in brackets for context
4. **Dynamic Pricing** - Prices can be updated after creation with `update_deliverable_price()`
5. **Accurate Totals** - All balance sheet totals are meaningful monetary sums

This implementation fully solves GitHub issue #3 by requiring monetary valuation for all deliverables, ensuring balance sheets maintain proper accounting principles.